# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [ ]:
! pip install seaborn opencc matplotlib scikit-learn tqdm

In [2]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

In [ ]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

In [4]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [ ]:
char_to_id = {}
id_to_char = {}

# Add special tokens first
char_to_id['<pad>'] = 0  
char_to_id['<eos>'] = 1

# Add common characters from expressions
chars = set()
for row in df_train['src'].values:
    chars.update(list(row))

# Sort chars to ensure consistent ordering
sorted_chars = sorted(list(chars))
for char in sorted_chars:
    if char not in char_to_id:
        idx = len(char_to_id)
        char_to_id[char] = idx

# Create reverse mapping
for char, idx in char_to_id.items():
    id_to_char[idx] = char

vocab_size = len(char_to_id)
print(f'Vocab size: {vocab_size}')

# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


In [ ]:
def preprocess_sequence(df):
    df['char_id_list'] = df['src'].apply(lambda x: [char_to_id[c] for c in x] + [char_to_id['<eos>']])
    
    # For each sequence, create label list with <pad> before '=' and actual digits after
    def create_label_sequence(row):
        # Find index of '=' in the input sequence
        eq_idx = row['src'].find('=')
        # Before = should be <pad>
        prefix = [char_to_id['<pad>']] * eq_idx
        # After = should be target number + <eos>
        suffix = [char_to_id[c] for c in row['tgt']] + [char_to_id['<eos>']]
        return prefix + suffix
    
    df['label_id_list'] = df.apply(create_label_sequence, axis=1)
    return df

df_train = preprocess_sequence(df_train) 
df_eval = preprocess_sequence(df_eval)

df_train.head()

# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [7]:
batch_size = 256
epochs = 10
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, index):
        x = self.sequences.iloc[index]['char_id_list']
        y = self.sequences.iloc[index]['label_id_list']
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [9]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']])

In [10]:
# Build dataloader of train set and eval set, collate_fn is the collate function
# Create dataloaders
dl_train = torch.utils.data.DataLoader(
    ds_train, 
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

dl_eval = torch.utils.data.DataLoader(
    ds_eval,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [11]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                         embedding_dim=embed_dim,
                                         padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                      hidden_size=hidden_dim,
                                      batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                      hidden_size=hidden_dim,
                                      batch_first=True)
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    def encoder(self, x, x_lens):
        # 確保輸入是二維的 [batch_size, seq_len]
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        if not isinstance(x_lens, torch.Tensor):
            x_lens = torch.tensor([x_lens])
        if len(x_lens.shape) == 0:
            x_lens = x_lens.unsqueeze(0)
            
        x = self.embedding(x)
        
        x = torch.nn.utils.rnn.pack_padded_sequence(
            x,
            x_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )
        
        x, _ = self.rnn_layer1(x)
        x, _ = self.rnn_layer2(x)
        
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(
            x,
            batch_first=True
        )
        
        x = self.linear(x)
        return x

    def generator(self, expression, max_len=200):
        """
        Improved generator that handles a single expression
        
        Args:
            expression: String containing the expression to evaluate
            max_len: Maximum length of generated sequence
        """
        device = next(self.parameters()).device
        
        # Find position of equals sign
        eq_position = expression.find('=')
        if eq_position == -1:
            return expression
            
        # Convert expression to character IDs
        char_ids = [char_to_id[c] for c in expression]
        current_position = eq_position + 1
        
        # Initialize the input tensor
        x = torch.tensor(char_ids, device=device)
        x = x.unsqueeze(0)  # Add batch dimension
        
        # Generate the answer
        generated_chars = []
        
        while current_position < max_len:
            with torch.no_grad():
                # Get model output
                x_len = torch.tensor([len(x[0])], device=device)
                output = self.encoder(x, x_len)
                
                # Get next character prediction
                logits = output[0, current_position - 1]
                next_char = torch.argmax(logits).item()
                
                # Check for end of sequence
                if next_char == char_to_id['<eos>']:
                    break
                    
                # Add to generated sequence
                generated_chars.append(next_char)
                
                # Update input for next iteration
                if current_position < len(x[0]):
                    x[0, current_position] = next_char
                else:
                    x = torch.cat([x, torch.tensor([[next_char]], device=device)], dim=1)
                
                current_position += 1

        # Convert result back to string
        result = expression[:eq_position + 1]
        result += ''.join(id_to_char[idx] for idx in generated_chars 
                         if idx not in [char_to_id['<eos>'], char_to_id['<pad>']])
        
        return result

In [12]:
torch.manual_seed(2)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

In [13]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [14]:
from tqdm import tqdm
import math

def validate_model(model, dataloader, device):
    """
    Validates the model one expression at a time to ensure maximum accuracy
    
    Args:
        model: The model to validate
        dataloader: DataLoader containing validation data
        device: Device to run the model on
    """
    model.eval()
    matched = 0
    total = 0
    
    with torch.no_grad():
        bar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Validation")
        
        for _, (batch_x, batch_y, batch_x_lens, batch_y_lens) in bar:
            # Process each example in the batch individually
            for i in range(batch_x.size(0)):
                # Extract single expression
                expr_tokens = [id_to_char[idx.item()] for idx in batch_x[i] 
                             if idx.item() not in [char_to_id['<pad>'], char_to_id['<eos>']]]
                expr = ''.join(expr_tokens)
                eq_idx = expr.find('=')
                
                if eq_idx != -1:
                    # Get input expression up to equals sign
                    input_expr = expr[:eq_idx + 1]
                    
                    # Get true answer
                    true_tokens = [id_to_char[idx.item()] for idx in batch_y[i]
                                if idx.item() not in [char_to_id['<pad>'], char_to_id['<eos>']]]
                    true_answer = ''.join(true_tokens)
                    
                    # Generate prediction for single expression
                    try:
                        prediction = model.generator(input_expr)
                        eq_idx = prediction.find('=')
                        if eq_idx != -1:
                            pred_answer = prediction[eq_idx + 1:].strip()
                            true_answer = true_answer.strip()
                            
                            if pred_answer == true_answer:
                                matched += 1
                            total += 1
                            
                            # Update progress bar
                            if total > 0:
                                bar.set_postfix({
                                    'accuracy': f"{matched/total:.4f}",
                                })
                    except Exception as e:
                        print(f"Error processing expression {input_expr}: {str(e)}")
                        continue
    
    final_accuracy = matched/total if total > 0 else 0
    print(f"\nFinal Accuracy: {final_accuracy:.4f}")
    print(f"Correct: {matched}/{total}")
    return final_accuracy

# Training loop
def train_model(model, train_loader, eval_loader, criterion, optimizer, 
                device, epochs, grad_clip):
    
    for epoch in range(1, epochs + 1):
        model.train()
        bar = tqdm(train_loader, desc=f"Train epoch {epoch}")
        
        for i, (batch_x, batch_y, batch_x_lens, batch_y_lens) in enumerate(bar):
            optimizer.zero_grad()
            
            # Forward pass
            batch_pred_y = model(batch_x.to(device), batch_x_lens)
            batch_pred_y = batch_pred_y[:, :batch_y.size(1), :].contiguous().view(-1, vocab_size)
            batch_y = batch_y.to(device).contiguous().view(-1)
            
            # Calculate loss and backpropagate
            loss = criterion(batch_pred_y, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            
            # Update progress bar every 50 iterations
            if i % 50 == 0:
                bar.set_postfix(loss=loss.item())
        
    
    return model

In [ ]:
model = model.to(device)
model = train_model(
    model=model,
    train_loader=dl_train,
    eval_loader=dl_eval,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=epochs,
    grad_clip=grad_clip
)

In [20]:
score = validate_model(model, dl_eval, device)
print(f'Final Score: {score}')

# Save Model

In [108]:
# Save model
import time

date = time.strftime("%Y%m%d")
torch.save(model.state_dict(), f'{date}_{score}.pth')

# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [ ]:
# Load model
model_name = '20241113_0.2357037037037037.pth'
model = CharRNN(vocab_size, embed_dim, hidden_dim)
model.load_state_dict(torch.load(model_name))

In [ ]:
model = model.to('cpu')

print("".join(model.generator('10*35*26=')))